# Oxidamp Playground

## Prerequistites

To execute and explore this notepad requires [evcxr_jupyter](https://github.com/google/evcxr/tree/main/evcxr_jupyter). To install this on a new machine try:

    rustup component add rust-src
    sudo apt install jupyter-notebook cmake build-essential
    cargo install evcxr_jupyter
    evcxr_jupyter --install
    jupyter-notebook
    
For more details checkout the [excxr documentation](https://github.com/google/evcxr/tree/main/evcxr_jupyter).

## Initial setup
Before we can do anything else, we must compile the oxidamp crate. Note that we have set the path to `"."` and this requires that `jupyter-notebook` is started from the oxidamp directory.

In [2]:
:dep oxidamp = { path="." }
use oxidamp::*;

Create a new audio context to configure things against:

In [3]:
let ctx = AudioContext::new(48000);

## Making audio playable within the jupyter notebook

This works really well to allow interactive experiments with the oxidamp DSP code. However it will also significantly bloat the git repo for every audio sample we include because the audio it generates is uncompressed and ends up burned into the notebook file.

Use with both taste and care!

In [4]:
:dep base64
:dep vorbis-encoder

const QUALITY: f32 = 3.0;

pub struct AudioClip {
    content: String
}

impl AudioClip {
    fn from_mono(buf: &[f32], sample_rate: u64) -> Self {
        let mut encoder = vorbis_encoder::Encoder::new(1, sample_rate, QUALITY).unwrap();
        let input: Vec<_> = buf.iter().map(|&x| (x * 32767.0) as i16).collect();
        let mut output = encoder.encode(&input).unwrap();
        output.append(&mut encoder.flush().unwrap());        

        Self { content: base64::encode(output) }
    }
    
    fn from_stereo(lbuf: &[f32], rbuf: &[f32], sample_rate: u64) -> Self {
        let mut encoder = vorbis_encoder::Encoder::new(2, sample_rate, QUALITY).unwrap();
        let lchan = lbuf.iter().map(|&x| (x * 32767.0) as i16);
        let rchan = rbuf.iter().map(|&x| (x * 32767.0) as i16);
        let mut input = Vec::new();
        for (l, r) in lchan.zip(rchan) {
            input.push(l);
            input.push(r);
        }
        let mut output = encoder.encode(&input).unwrap();
        output.append(&mut encoder.flush().unwrap()); 
        
        Self { content: base64::encode(output) }
    }
    
    pub fn evcxr_display(&self) {
        println!("EVCXR_BEGIN_CONTENT text/html\n<audio controls src=\"data:application/ogg;base64,{}\"/>\nEVCXR_END_CONTENT", self.content);
    }
}

version = "3.0", features = ["derive"] }
hound = "3.version = "3.0", features = ["derive"] }
hound = "3.### Minimal example: Two seconds of noise

In [5]:
fn randbuf<const L: usize>() -> [f32; L] {
    let mut seed = 1;
    let mut inbuf = [0.0_f32; L];
    for i in &mut inbuf {
        *i = frand31(&mut seed);
    }
    inbuf
}
let noise = randbuf::<88200>();

AudioClip::from_mono(&noise, 44100)

# Examples

## Cabinet Simulation

Oxidamp includes a speaker cabinet simulator. Currently it is modelled (not very accurately) on the frequency reponse of a classic speaker. Implemented as a combination of biquad filters.

In [6]:
let mut cabsim = CabinetSimulator::default();
cabsim.setup(&ctx);

We can use the filter `stimulate()` method to get the frequency response for an 400Hz sine wave:

In [7]:
linear2db(cabsim.stimulate(&ctx, 400))

-13.406087

## Karplus-Strong synthesis

In [8]:
let mut synth = KarplusStrong::default();
let mut snd = [0.0_f32; 2*48000];

synth.setup(&ctx);
synth.trigger();
synth.process(&mut snd);

AudioClip::from_mono(&snd, 48000)

## Signal Generation

In [9]:
let mut sg = SineGenerator::default();
sg.setup(&ctx, 440, 0.7);
let mut sine = [0.0_f32; 96000];
sg.process(&mut sine);

## Drum Machine

The oxidamp drum machine is deliberately lo-fi. It is based on 8-bit samples clocking in at 23Khz. However to make it less robotic the resampling filters change the parameters every sample.

Try it, you might like it.

In [10]:
let mut dm = DrumMachine::default();
dm.setup(&ctx);
dm.set_control(&Control::BeatsPerMinute(112));
dm.set_control(&Control::Pattern(3));

let mut jcrev = Reverb::default();

let mut dry = [0.0_f32; 10*48000];
let mut wet = [0.0_f32; 10*48000];

dm.process(&mut dry);
jcrev.process(&dry, &mut wet);

AudioClip::from_mono(&dry, 48000)

... and mixed with a little reverb:

In [11]:
for (w, d) in wet.iter_mut().zip(dry.iter()) {
    *w = *d + *w * 0.33
}

AudioClip::from_mono(&wet, 48000)